This jupyter notebook has been created to show [Folium] library capabilities with maps. 

Folium makes it easy to visualize data that’s been manipulated in Python on an interactive [Leaflet.js] map.

It is possible to interact with this jupyter notebook using [nbviewer] in the following link:

http://nbviewer.jupyter.org/github/alvertogit/datascience/blob/master/Map%20examples%20with%20Folium.ipynb

[Folium]: https://github.com/python-visualization/folium
[Leaflet.js]: https://leafletjs.com/
[nbviewer]: http://nbviewer.jupyter.org/

In [1]:
import folium
import pandas as pd # data structures
import numpy as np # linear algebra, arrays

Countries selection on World Map

In [2]:
# world countries boundaries json file
world_countries = "https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/world-countries.json"

import urllib.request, json 
with urllib.request.urlopen(world_countries) as url:
    geo_json_data = json.loads(url.read().decode())

    
def country_color_function(feature):
    """Countries which id start with 'A' set to green and others to grey."""
    if str(feature['id']).startswith('A'):
        return '#00f000'
    else:
        return '#1f0000'
    
world_map = folium.Map(location=[0.0, 0.0],
                       tiles='cartodbpositron',
                       zoom_start=2)

folium.GeoJson(
    world_countries,
    name='geojson',
    style_function=lambda feature: {
        'fillColor': country_color_function(feature),
        'color': 'black',
        'weight': 2,
        'dashArray': '5, 5'
    }
).add_to(world_map)


world_map

Markers on Map

In [3]:
from folium.plugins import MarkerCluster

madrid_map = folium.Map(location=[40.4169, -3.7033],
                        tiles='stamenterrain',
                        zoom_start=11)

colors_list = ['red', 'blue', 'green', 'orange', 'purple', 'darkred',
               'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue',
               'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen',
               'gray', 'black', 'lightgray']

colors_len = len(colors_list)

# markers
marker_keys = ['name','location','icon']

airport = ['Adolfo Suarez Madrid Barajas Airport',[40.47, -3.57],'plane']
station = ['Madrid Puerta de Atocha Station',[40.406, -3.69],'ok-sign']
home = ['Home',[40.5, -3.66],'home']
museum = ['Museum',[40.44, -3.73],'remove-sign']

markers_list = []
markers_list.append(dict(zip(marker_keys,airport)))
markers_list.append(dict(zip(marker_keys,station)))
markers_list.append(dict(zip(marker_keys,home)))
markers_list.append(dict(zip(marker_keys,museum)))

# create marker cluster added to the map
marker_cluster = MarkerCluster().add_to(madrid_map)

# add markers to cluster
for i,marker in enumerate(markers_list):
    folium.Marker(
        location=marker['location'],
        popup=marker['name'],
        icon=folium.Icon(color=colors_list[i%colors_len],
                         icon=marker['icon']),
    ).add_to(marker_cluster)

# add table to popup
df = pd.DataFrame(data=[['09:00 to 19:00', '10:00 to 17:00'], ['10:00 to 18:00', '10:00 to 15:00']], 
                  columns=['High Season Opening Hours', 'Low Season Opening Hours'], 
                  index=['Monday to Friday','Weekends & Holidays'])

html = df.to_html(classes='table table-striped table-hover table-condensed table-responsive')

popup_table = folium.Popup(html)

folium.Marker(
    location=[40.485, -3.74],
    popup=popup_table,
    icon=None,
).add_to(marker_cluster)

madrid_map

Heat Map generated with random data

In [4]:
data = (np.random.normal(size=(50, 3)) *
        np.array([[5, 5, 5]]) +
        np.array([[47, 20, 1]])).tolist()

In [5]:
from folium.plugins import HeatMap

europe_map = folium.Map([47, 2], 
                        tiles='stamentoner', 
                        zoom_start=4)

HeatMap(data).add_to(europe_map)

europe_map

Lines and circles on Map

In [6]:
line_keys = ['stations','points']

seville_paris = [['Paris','Barcelona','Madrid','Seville'],
                 [[48.858093, 2.294694],
                  [41.390205, 2.154007],
                  [40.4169, -3.7033],
                  [37.392529, -5.994072]]]

paris_brussels = [['Brussels','Paris'],
                  [[50.8333, 4.35],
                   [48.858093, 2.294694]]]

brussels_berlin = [['Berlin','Brussels'],
                   [[52.5186, 13.4081],
                    [50.8333, 4.35]]]

paris_london = [['Paris','Calais','London'],
                [[51.5072, -0.1275],
                 [50.9475, 1.91],
                 [48.858093, 2.294694]]]

lines_list = []
lines_list.append(dict(zip(line_keys,paris_brussels)))
lines_list.append(dict(zip(line_keys,seville_paris)))
lines_list.append(dict(zip(line_keys,brussels_berlin)))
lines_list.append(dict(zip(line_keys,paris_london)))

# create the map
lines_map = folium.Map(location=[46, 0], 
                       zoom_start=5)

# create feature group
fg = folium.FeatureGroup(name='My Lines Map')

# add lines to feature group
for i,line in enumerate(lines_list):
    my_PolyLine=folium.PolyLine(color=colors_list[i%colors_len],
                                locations=line['points'],
                                weight=5)
    fg.add_child(my_PolyLine)

    # add circles to feature group
    for pos in range(len(line['points'])):
        my_Circle=folium.CircleMarker(location=line['points'][pos],
                                      radius=6,
                                      popup=line['stations'][pos],
                                      fill=True,
                                      fill_color='white',
                                      color='grey',
                                      fill_opacity=0.7)
        fg.add_child(my_Circle)


    
# add the feature group to the map
lines_map.add_child(fg)

Great Circles on Map

In [7]:
import math

def gcIntermediatePoints(origin,destination,steps):
    """Calculate Intermediate Points of Great Circle.
    
    Formula:
    
    f: factor
    d: distance between points
    
    A = sin((1 - f) * d) / sin(d)
    B = sin(f * d) / sin(d)
    x = A * cos(lat1) * cos(lon1) + B * cos(lat2) * cos(lon2)
    y = A * cos(lat1) * sin(lon1) + B * cos(lat2) * sin(lon2)
    z = A * sin(lat1) + B * sin(lat2)
    lat = atan2(z, sqrt(x ^ 2 + y ^ 2))
    lon = atan2(y, x)
    
    Source http://www.edwilliams.org/avform.htm
    
    Args:
        origin [Lat,Lon]: Origin point coordinates in degrees.
        destination [Lat,Lon]: Destination point coordinates in degrees.
        steps (int): Number of steps between origin and destination.
    
    Returns:
        gcPoints: Return great circle points including origin and destination.
        
    """
    
    ori_lat,ori_long = origin
    dest_lat,dest_long = destination
    
    ori_lat = math.radians(ori_lat)
    ori_long = math.radians(ori_long)
    dest_lat = math.radians(dest_lat)
    dest_long = math.radians(dest_long)
    
    distance = 2 * math.asin(
        math.sqrt(math.pow((math.sin((ori_lat - dest_lat)/2)),2) +
        math.cos(ori_lat) * math.cos(dest_lat) * math.pow(math.sin((ori_long-dest_long)/2),2)))
    
    gcPoints = [origin]
    
    # intermediate points
    for step in range(1,steps):
        
        fraction = float(step/steps)
    
        A = math.sin((1 - fraction) * distance) / math.sin(distance);
        B = math.sin(fraction * distance) / math.sin(distance);
        x = A * math.cos(ori_lat) * math.cos(ori_long) + B * math.cos(dest_lat) * math.cos(dest_long)
        y = A * math.cos(ori_lat) * math.sin(ori_long) + B * math.cos(dest_lat) * math.sin(dest_long)
        z = A * math.sin(ori_lat) + B * math.sin(dest_lat)
        point_lat = math.atan2(z, math.sqrt(math.pow(x, 2) + math.pow(y, 2)))
        point_lng = math.atan2(y, x)
        
        gcPoints.append([math.degrees(point_lat),math.degrees(point_lng)])
    
    gcPoints.append(destination)
    
    return gcPoints

In [8]:
gc_map = folium.Map(location=[44.0, -30.0],
                    tiles='cartodbpositron',
                    zoom_start=3)

city_keys = ['name', 'location']
ny_data = ['New York',[40.488224,-73.131836]]
paris_data = ['Paris',[48.856667,2.350987]]
madrid_data = ['Madrid',[40.4169, -3.7033]]
london_data = ['London',[51.5072, -0.1275]]
berlin_data = ['Berlin',[52.5186, 13.4081]]

ny = dict(zip(city_keys,ny_data))
paris = dict(zip(city_keys,paris_data))
madrid = dict(zip(city_keys,madrid_data))
london = dict(zip(city_keys,london_data))
berlin = dict(zip(city_keys,berlin_data))

cities_list = [ny,paris,madrid,london,berlin]

# great circles
ny_paris_gc = gcIntermediatePoints(ny['location'],paris['location'],10)
ny_madrid_gc = gcIntermediatePoints(ny['location'],madrid['location'],10)
ny_london_gc = gcIntermediatePoints(ny['location'],london['location'],10)
ny_berlin_gc = gcIntermediatePoints(ny['location'],berlin['location'],10)

gc_list = []
gc_list.append(ny_paris_gc)
gc_list.append(ny_madrid_gc)
gc_list.append(ny_london_gc)
gc_list.append(ny_berlin_gc)

# create feature group
fg = folium.FeatureGroup(name='My Great Circle Lines Map')

# add great circles to feature group
for i,gc in enumerate(gc_list):
    my_PolyLine=folium.PolyLine(color=colors_list[i%colors_len],
                                locations=gc,
                                weight=5)
    fg.add_child(my_PolyLine)

# add circles to feature group
for city in cities_list:
    my_Circle=folium.CircleMarker(location=city['location'],
                        radius=6,
                        popup=city['name'],
                        fill=True,
                        fill_color='white',
                        color='grey',
                        fill_opacity=0.7)
    fg.add_child(my_Circle)

# add feature group to map
gc_map.add_child(fg)